<a href="https://colab.research.google.com/github/Takumi173/JPMA2023TF1-1/blob/main/JPMA2023_case2_(2)_LLM%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E3%83%A1%E3%83%83%E3%82%BB%E3%83%BC%E3%82%B8%E3%81%AE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続とパスの指定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# データが保存されたディレクトリの指定
datadir = '/content/drive/MyDrive/JPMA2023_case2/'

Mounted at /content/drive


## ライブラリとパッケージのインストール

In [2]:
!pip install transformers bitsandbytes accelerate

import pickle
import torch
import gc
import pandas as pd
import warnings
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm.notebook import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 38.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

## LLMモデルのロード

In [3]:
# HuggingFaceのログイン
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# 使用するモデルの指定

model_id = 'meta-llama/Llama-2-13b-chat-hf'
#model_id = 'codellama/CodeLlama-13b-Instruct-hf'
#model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

In [5]:
# Tokenizerのロード
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# モデルごとのbnb config設定
if model_id == 'mistralai/Mistral-7B-Instruct-v0.2':
  # Mistral 7Bは8bitでロード
  bnb_config = BitsAndBytesConfig(
      load_in_8bit=True,
      )
else:
  # Llama2/CodeLlamaは13Bなので4bitでロード
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.float16,
      )

# モデルのロード
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=bnb_config
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# データの前処理

In [6]:
# 生成対象データの読み込み

df = pd.read_csv(datadir + 'Split_20_Test.csv').drop('Message',axis=1)
df

,Code
0,"[Lab Test or Examination Name] = ""Erythrocytes..."
1,"[Lab Test or Examination Name] = ""Leukocytes""\..."
2,[Route] IS EMPTY \nAND \n[Are there any previo...
3,[ECG Performed Time] > [Dosing Time] + 60 or [...
4,[Diameter] IS EMPTY \nAND \n[TOO SMALL TO MEAS...
...,...
122,"[Lab Test or Examination Name] = ""Triglyceride..."
123,[A] > 8
124,"[Test Result] = ""POSITIVE"""
125,"[Lab Test or Examination Name] = ""Specific Gra..."


In [7]:
# Pseudo codeの表記ゆれ吸収

def operator_standardizer(Text):
  # -+*/<>^=!の前後に空白を追加
  t = re.sub(r'([-+*/<>^=!])', r' \1 ', Text)

  # 空白の連続を除去
  t = re.sub('\s+', ' ', t)

  # -+*/<>^=!に挟まれた空白の除去
  t = re.sub(r'([-+*/<>^=!])\s([-+*/<>^=!])', r'\1\2', t)

  # アルファベットに挟まれた-の前後の空白を削除
  t = re.sub(r'([a-zA-Z])\s-\s([a-zA-Z])', r'\1-\2', t)

  # アルファベットに挟まれた/の前後の空白を削除
  t = re.sub(r'([a-zA-Z])\s/\s([a-zA-Z])', r'\1/\2', t)

  #表現統一
  t = re.sub('=<','<=', t)
  t = re.sub('=>','>=', t)
  t = re.sub('<>','!=', t)
  t = re.sub('\^=','!=', t)
  return t

df['Code_'] = df['Code'].apply(operator_standardizer)
df.query('Code != Code_')

,Code,Code_
0,"[Lab Test or Examination Name] = ""Erythrocytes...","[Lab Test or Examination Name] = ""Erythrocytes..."
1,"[Lab Test or Examination Name] = ""Leukocytes""\...","[Lab Test or Examination Name] = ""Leukocytes"" ..."
2,[Route] IS EMPTY \nAND \n[Are there any previo...,[Route] IS EMPTY AND [Are there any previous m...
4,[Diameter] IS EMPTY \nAND \n[TOO SMALL TO MEAS...,[Diameter] IS EMPTY AND [TOO SMALL TO MEASURE]...
5,[A] =< 7,[A] <= 7
...,...,...
117,"[Lab Test or Examination Name] = ""Lipase""\nAND...","[Lab Test or Examination Name] = ""Lipase"" AND ..."
118,"[Lab Test or Examination Name] = ""Creatine Kin...","[Lab Test or Examination Name] = ""Creatine Kin..."
119,[Date of Visit] = null \nor \n[Date of Visit] ...,[Date of Visit] = null or [Date of Visit] < [S...
122,"[Lab Test or Examination Name] = ""Triglyceride...","[Lab Test or Examination Name] = ""Triglyceride..."


# メッセージの生成

## 関数定義

In [8]:
# GPU memory flusher
def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  return

# Generator
def Generator (Code):
  flush()

  prompt = f"""<s>[INST] <<SYS>>
You are a helpful, respectful, honest and excellent programmer.  Always answer as helpfully as possible, while being safe.  Please answer the questions as accurately as possible based on the knowledge you have.  Take a deep breath, read the text carefully, and think step-by-step!
<</SYS>>

### CONTEXT
Here are examples of translations from CODE to TEXT.

Example 1:
CODE: [Overall Response] != "PROGRESSIVE DISEASE" AND [Overall Response] IS NOT EMPTY AND [New Lesion Progression] = "UNEQUIVOCAL"
TEXT: The [Overall Response] is not "PROGRESSIVE DISEASE", yet the [New Lesion Progression] is "UNEQUIVOCAL".  Please review.

Example 2:
CODE: [Stop Date] > [Date of First Study Drug Taken] - 30
TEXT: The [Stop Date] is later than 30 days prior to the [Date of First Study Drug Taken].  Please review.

Example 3:
CODE: [Primary Study Drug Treatment Status] = "COMPLETED" and [Date of Last Dose of Study Drug] < [Study Day 90]
TEXT: The [Primary Study Drug Treatment Status] is "COMPLETED", yet the [Date of Last Dose of Study Drug] is prior to the [Study Day 90].  Please review.

Example 4:
CODE: 18 > [Age]
TEXT: The [Age] is less than 18 years.

Example 5:
CODE: [LBTEST] = "Leukocytes" AND ([LBORRES] < 13 or [LBORRES] > 294)
TEXT: The [LBTEST] is "Leukocytes" and the [LBORRES] is not within the expected range of 13-294.  Please review.

Example 6:
CODE: Record count of [Subject ID] is more than 1
TEXT: Duplicate [Subject ID].  Please review.

Example 7:
CODE: [Actual Sample Time for Scheduled Time TIME 2] - 60 < [Actual Sample Time for Scheduled Time TIME 1]
TEXT: The [Actual Sample Time for Scheduled Time TIME 2] is less than 60 minutes after the [Actual Sample Time for Scheduled Time TIME 1].  Please review.

Example 8:
CODE: [Systolic Blood Pressure] >= 90 AND [Systolic Blood Pressure] <= 160 AND [Diastolic Blood Pressure] >= 50 AND [Diastolic Blood Pressure] <= 90 AND [Exclusion Criterion Affecting Eligibility] = "EX01"
TEXT: The [Systolic Blood Pressure] is greater than or equal to 90 and less than or equal to 160 and the [Diastolic Blood Pressure] is greater than or equal to 50 and less than or equal to 90 and the [Exclusion Criterion Affecting Eligibility] is equal to "EX01".  Please review.

Example 9:
CODE: [ECG Performed Time] > [Dosing Time] + 15 or [Time ECG Performed] < [Dosing Time]
TEXT: The [ECG Performed Time] is not within 15 minutes prior to the [Dosing Time].  Please review.

Example 10:
CODE: [Number of Bone Lesions] IS NOT EMPTY AND [Target Lesions] != "BONE LESION"AND [Target Lesions] IS NOT EMPTY
TEXT: The [Number of Bone Lesions] is provided yet the [Target Lesions] is not a "Bone Lesion".  Please review.

### QUESTION

Step1: Referring to the CONTEXT examples, translate the following CODE into TEXT exactly.  It is important to maintain the CODE and the TEXT square brackets or quotation marks around the words when translating.  Answer should be start with "TEXT:" and the TEXT should be end with "Please review."  No explanation required.
CODE: {Code}

Step2: Check if the text enclosed in square brackets or quotation marks in the CODE and the TEXT match exactly.  Also check that the logic of the CODE and the TEXT match exactly.  If not, modify the TEXT to match.


### ANSWER
[/INST] """

  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    # メモリ上限を超えないためのトークン数キャップ
    if len(token_ids[0]) > 1500:
      output = "Too many text to read"
      return prompt, output

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,   # 生成するメッセージの最大トークン数
        do_sample=False,      # ランダム生成をしない
    )

  # 生成メッセージの取得
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

  return prompt, output   # プロンプトと生成メッセージを返す

## テスト生成

In [9]:
#Code = '[Interpretation] = null and [Form Not Done] = null and [Not Done for this log line] = null'
#Code = '[Per your local regulations, is it allowed to collect a PARTIAL "Birth Date"?] = "NO" AND [Per your local regulations, is it allowed to collect a PARTIAL "Birth Date"?] IS NOT EMPTY AND [Self-Reported Race] <> "NOT REPORTED"'
Code = '[Lab Test or Examination Name] = "Lipase" and [Result] > 55 and [Reference Range Indicator]^= "H"'
#Code = '[A] - [B] =< 6'

prompt, output = Generator(Code)

print("*** OUTPUT ***")
print(output)
print("\n*** PROMPT ***")
print(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


*** OUTPUT ***
Sure, I'd be happy to help! Here are the translations for the CODE you provided:

Step 1:

TEXT: The [Lab Test or Examination Name] is "Lipase" and the [Result] is greater than 55 and the [Reference Range Indicator] is equal to "H". Please review.

Step 2:

The text enclosed in square brackets or quotation marks in the CODE and the TEXT match exactly. The logic of the CODE and the TEXT also match exactly. Therefore, there is no need to modify the TEXT.

*** PROMPT ***
<s>[INST] <<SYS>>
You are a helpful, respectful, honest and excellent programmer.  Always answer as helpfully as possible, while being safe.  Please answer the questions as accurately as possible based on the knowledge you have.  Take a deep breath, read the text carefully, and think step-by-step!
<</SYS>>

### CONTEXT
Here are examples of translations from CODE to TEXT.

Example 1:
CODE: [Overall Response] != "PROGRESSIVE DISEASE" AND [Overall Response] IS NOT EMPTY AND [New Lesion Progression] = "UNEQUIVO

## 評価データを用いたメッセージ生成

In [10]:
# Warningを表示しない
warnings.simplefilter('ignore')

ResList = []
PredList = []

for t in tqdm(df['Code_'].tolist()):
  _, Response = Generator(t)
  PredictedText = Response.split("\nTEXT:")[-1].split("\n")[0].strip()

  print(f'Generated *** {PredictedText}')
  ResList.append(Response)
  PredList.append(PredictedText)

df_pred = df
df_pred['Generator'] = model_id
df_pred['Response'] = ResList
df_pred['GeneratedText'] = PredList

df_pred

  0%|          | 0/127 [00:00<?, ?it/s]

Generated *** The [Lab Test or Examination Name] is "Erythrocytes" and the [Result] is less than 427, yet the [Reference Range Indicator] is not "L". Please review.
Generated *** The [Lab Test or Examination Name] is "Leukocytes" and the [Result] is greater than 98, yet the [Reference Range Indicator] is not "H". Please review.
Generated *** The [Route] is empty and the [Are there any previous medications and/or concomitant medications to record?] is "YES". Please review.
Generated *** The [ECG Performed Time] is not within 60 minutes prior to the [Dosing Time]. Please review.
Generated *** The [Diameter], [TOO SMALL TO MEASURE], [Not Done], and [Target Lesions] are all empty. Please review.
Generated *** The [A] is less than or equal to 7. Please review.
Generated *** The [Lab Test or Examination Name] is "Lipase" and the [Result] is greater than 55, yet the [Reference Range Indicator] is not "H". Please review.
Generated *** The [Lab Test or Examination Name] is "Amylase" and the [Re

,Code,Code_,Generator,Response,GeneratedText
0,"[Lab Test or Examination Name] = ""Erythrocytes...","[Lab Test or Examination Name] = ""Erythrocytes...",meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...","The [Lab Test or Examination Name] is ""Erythro..."
1,"[Lab Test or Examination Name] = ""Leukocytes""\...","[Lab Test or Examination Name] = ""Leukocytes"" ...",meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...","The [Lab Test or Examination Name] is ""Leukocy..."
2,[Route] IS EMPTY \nAND \n[Are there any previo...,[Route] IS EMPTY AND [Are there any previous m...,meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here's the transla...",The [Route] is empty and the [Are there any pr...
3,[ECG Performed Time] > [Dosing Time] + 60 or [...,[ECG Performed Time] > [Dosing Time] + 60 or [...,meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...",The [ECG Performed Time] is not within 60 minu...
4,[Diameter] IS EMPTY \nAND \n[TOO SMALL TO MEAS...,[Diameter] IS EMPTY AND [TOO SMALL TO MEASURE]...,meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here's the answer ...","The [Diameter], [TOO SMALL TO MEASURE], [Not D..."
...,...,...,...,...,...
122,"[Lab Test or Examination Name] = ""Triglyceride...","[Lab Test or Examination Name] = ""Triglyceride...",meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...","The [Lab Test or Examination Name] is ""Triglyc..."
123,[A] > 8,[A] > 8,meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...",The [A] is greater than 8. Please review.
124,"[Test Result] = ""POSITIVE""","[Test Result] = ""POSITIVE""",meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...","The [Test Result] is ""POSITIVE"". Please review."
125,"[Lab Test or Examination Name] = ""Specific Gra...","[Lab Test or Examination Name] = ""Specific Gra...",meta-llama/Llama-2-13b-chat-hf,"Sure, I'd be happy to help! Here are the trans...","The [Lab Test or Examination Name] is ""Specifi..."


In [11]:
# 結果の保存
df_pred.to_csv(datadir + model_id.split("/")[-1] +'_Generated_Massages.csv')